## Phase I: Preprocessing
This includes filtering .fastq files, extracting sequences, trimming primers, and generating a "tall" form CSV file (ie. the same variant appears in multiple lines, for different datasets, prefixes, etc). 

In [ ]:
#using Pkg
#Pkg.add(["YAML", "PyPlot", "DataFrames", "CSV", "NearestNeighbors", "LightGraphs", "LazySets", "Distances", "SparseArrays"])
#Pkg.add(PackageSpec(;name="SeqUMAP",url="https://github.com/murrellb/SeqUMAP.jl.git"))
#Pkg.add(PackageSpec(name="NextGenSeqUtils", rev="1.5.3", url = "https://github.com/MurrellGroup/NextGenSeqUtils.jl.git"))

using YAML, NextGenSeqUtils, PyPlot, StatsBase, DataFrames, CSV, SeqUMAP, NearestNeighbors, LightGraphs, LazySets, Distances, SparseArrays

include("../src/functions.jl")

In [ ]:
yaml = YAML.load_file("config.yaml")

In [ ]:
#Need to generalize this so that the yaml can specify multiple fwd and/or rev primers
#but this is fine for our current protocol
fwd_primer_options = uppercase.([yaml["primers"]["fwd"]])
rev_primer_options = uppercase.([yaml["primers"]["rev"]]);

min_fwd = min(20,minimum(length.(fwd_primer_options)))
min_rev = min(20,minimum(length.(rev_primer_options)))
short_fwd = [p[1:min_fwd] for p in fwd_primer_options]
short_rev = [p[1:min_rev] for p in rev_primer_options];

In [ ]:
fnames = [f["filename"] for f in yaml["files"]]
outputnames = [f["label"] for f in yaml["files"]];

filtered_path = "Filtered/"
processed_path = "Processed/"

required_directory_list = [filtered_path,processed_path,"Collected/","Lineages/","Logging/","Plots/","Plots/Correlations/","Selections/"]
for d in required_directory_list
    if !isdir(d)
        println("Directory $(d) must exist - creating it.")
        mkdir(d)
    end
end

for f in fnames
    if !isfile(f)
        @error "$(f), specified in the config file, is not present"
    end
end

In [ ]:
for i in 1:length(fnames)
    fname = fnames[i]
    outname = outputnames[i]
    println(fname," => ",outname)
    figure()
    length_vs_qual(fname,alpha = 0.005);
    savefig(filtered_path*outname*"_LvQ.png")
    fastq_filter(fname, filtered_path*outname*"_filt.fastq", error_rate = 0.01, min_length = 300);
    
    println("Reading filtered seqs")
    sleep(0.5)
    
    seqs,phreds,seqnames = read_fastq(filtered_path*outname*"_filt.fastq");
    
    println("Demuxing")
    sleep(0.5)
    demux_dic = demux_dict(seqs,short_fwd,short_rev,verbose=false,tol_one_error = true);
    
    println("Trimming")
    sleep(0.5)
    trimmed_seqs = []
    for k in keys(demux_dic)
        push!(trimmed_seqs,[double_primer_trim(seqpair[1],fwd_primer_options[k[1]],rev_primer_options[k[2]]) for seqpair in demux_dic[k]])
    end

    println("Collapsing and exporting")
    sleep(0.5)
    seqs = vcat(trimmed_seqs...);
    ka = collapse(seqs,prefix = outname*"_");
    write_fasta(processed_path*outname*".trimmed.fasta",ka[2],names = ka[1])
    write_fasta(processed_path*outname*".top500.trimmed.fasta",ka[2][1:500],names = ka[1][1:500])
end

proc_names = outputnames.*".trimmed.fasta";

In [ ]:
#barcode_list = grow(grow(grow(["T","G"],["A","T"]),["A","C","G","T"]),["A","C","G","T"])
barcode_list = grow(grow(["T","G"],["A","T"]),["A","C","G","T"])
#barcode_list = ["A","C","G","T"]
barcode2int = Dict(zip(barcode_list,1:length(barcode_list)));

In [ ]:
variants = []
for fname in proc_names[end:-1:1]
    println(fname)
    n,s = read_fasta_with_names(processed_path*fname)
    println(length(n))
    for i in 1:length(n)
        v = encode_variant(n[i],s[i])
        if v[5] != "bad" && v[6] #This is where we xclude non-functional variants.
            push!(variants,v)
        end
    end
end

In [ ]:
countmap([v[1] for v in variants])

In [ ]:
#dataset,count, barcode_num,prefix,hinge,functional,trimmed_AA_seq,outseq
df = DataFrame()
df.dataset = [v[1] for v in variants]
df.count = [v[2] for v in variants]
df.barcode_num = [v[3] for v in variants]
df.prefix = [v[4] for v in variants]
df.hinge = [v[5] for v in variants]
df.functional = [v[6] for v in variants]
df.trimmed_AA_seq = [v[7] for v in variants]
df.trimmed_nuc_seq = [v[8] for v in variants];

In [ ]:
alldatasets = sort(union(df.dataset))
alldatasets = alldatasets[sortperm(alldatasets .== yaml["baseline"], rev = true)]

dataset_counts = [sum(df.count[df.dataset .== d]) for d in alldatasets]
dataset2counts = Dict(zip(alldatasets,dataset_counts))

df.frequencies = [(df.count[i])/dataset2counts[df.dataset[i]] for i in 1:length(df.count)];
CSV.write("Collected/trimmed.csv",df);

## Phase II: Aggregating variants and calculating metrics
This takes the "tall" CSV, and collapses by variant (ignoring variation introduced by the cloning primer). It uses this cloning primer variation to create correlation plots that give a sense of how reliable the panning was. Random/failed panning will give worse correlations. It also maps CDRs

In [ ]:
df = CSV.read("Collected/trimmed.csv", DataFrame);

alldatasets = sort(union(df.dataset))
alldatasets = alldatasets[sortperm(alldatasets .== yaml["baseline"], rev = true)]
baseline_ind = findfirst(alldatasets .== yaml["baseline"]);

println("Baseline dataset is ",alldatasets[1])
dataset_counts = [sum(df.count[df.dataset .== d]) for d in alldatasets]
dataset2counts = Dict(zip(alldatasets,dataset_counts))

dataset2ind = Dict(zip(alldatasets,1:length(alldatasets)));
variant_master_list = sort(union(df.trimmed_nuc_seq))
variant2ind = Dict(zip(variant_master_list,1:length(union(df.trimmed_nuc_seq))));

num_barcodes = maximum(df.barcode_num);

In [ ]:
#variants, datasets, barcodes
#This is ones to get pseudocounts
count_matrix = zeros(length(variant2ind),length(dataset2ind),num_barcodes);

for i in 1:size(df)[1]
    v_ind = variant2ind[df.trimmed_nuc_seq[i]]
    dset = dataset2ind[df.dataset[i]]
    bcode = df.barcode_num[i]
    if bcode > 0
        count = df.count[i]
        count_matrix[v_ind,dset,bcode] += count
    end
end

#Matrix of frequencies over all "barcode" variants
freqmat = copy(count_matrix)
for i in 1:length(alldatasets)
    freqmat[:,i,:] ./= sum(freqmat[:,i,:])
end

#This is a log count matrix, regularized with a tiny constant.
logfreqmat = log10.(freqmat .+ 1/sum(dataset_counts));
enrich_mat = zeros(length(variant2ind),length(dataset2ind),num_barcodes);
for i in 1:length(alldatasets)
    enrich_mat[:,i,:] .= logfreqmat[:,i,:] .- logfreqmat[:,baseline_ind,:]
end;

#This the the marginal (ie. summed over different barcodes) freq and enrichment matrices.
#Freqs are regularized by 1/the number of variants.
num_vars = length(variant2ind);
marginal_freqs = sum(freqmat,dims = 3)[:,:];
#logmarginal_freqs = log10.(marginal_freqs .+ 1/sum(dataset_counts));
logmarginal_freqs = log10.(marginal_freqs .+ 1/num_vars);
logmarginal_enrich = logmarginal_freqs .- logmarginal_freqs[:,baseline_ind];

In [ ]:
triplet_inds = []
for i in 1:size(count_matrix)[1]
    tops = sortperm(count_matrix[i,baseline_ind,:],rev = true)[[1,2]]
    push!(triplet_inds,(i,tops))
end

In [ ]:
function plot_box()
    plot([0.0,3],[0.0,0.0],"--",color = "blue", alpha = 0.4)
    plot([0.0,0.0],[0.0,3],"--",color = "blue", alpha = 0.4)
    plot([1.5,3],[1.5,1.5],"--",color = "red", alpha = 0.4)
    plot([1.5,1.5],[1.5,3],"--",color = "red", alpha = 0.4)
end
lb,ub = -2.5,3.5
al = 0.6
comap = "rainbow"

In [ ]:
#starting_size_order = sortperm(sum(count_matrix[:,baseline_ind,:],dims = 2)[:],rev=true);
starting_size_order = sortperm(sum(count_matrix,dims = [2,3])[:],rev=true);
nu = 50000 #number to plot?
for i in 1:length(alldatasets)
    figure(figsize = (3,3))
    dset = i
    pairs = hcat([enrich_mat[i[1],:,i[2]][dset,:] for i in triplet_inds[starting_size_order[1:nu]]]...);
    #scatter(pairs[1,:],pairs[2,:],c=min_sizes,cmap = "rainbow")
    scatter(pairs[1,:],pairs[2,:],s = 1.0, c="black",cmap = comap,alpha = al, linewidth = 0.0)
    plot_box()
    xlim(lb,ub)
    ylim(lb,ub)
    xlabel(alldatasets[dset]*"a")
    ylabel(alldatasets[dset]*"b")
    tight_layout()
    savefig("Plots/Correlations/$(alldatasets[dset])_correlation.png", dpi = 450)
end

In [ ]:
figure(figsize = (5,1.5))
plt.hist(length.(variant_master_list),200:3:460);

In [ ]:
#Filtering out barely-observed sequences. This could maybe be more agressive.
min_thresh = yaml["min_count_thresh"]
marginal_counts = sum(count_matrix,dims = 3)[:,:];
keep_inds = (sum(marginal_counts,dims = 2)[:] .>= min_thresh) .& (length.(variant_master_list) .> 275) .& (length.(variant_master_list) .< 450)
sum(keep_inds)

In [ ]:
keep_variants = variant_master_list[keep_inds]
keep_AAs = robust_translate.(keep_variants);

In [ ]:
gaptrimmed = read_fasta(yaml["nanobody_MSA"])[2];
nanobodyprofile = seqs2profile(gaptrimmed);

In [ ]:
@time CDRs = CDRcut.(keep_AAs);

In [ ]:
region_names = ["CDR1","CDR2","CDR3"];
nuc_regions = [extract_nuc_regions(keep_variants[i],CDRs[i]) for i in 1:length(keep_variants)];
AA_regions = [extract_AA_regions(keep_AAs[i],CDRs[i]) for i in 1:length(keep_AAs)];

trimmed_keep_variants = [n[4] for n in nuc_regions];
trimmed_keep_AAs = [n[4] for n in AA_regions];

got_trimmed = length.(trimmed_keep_variants) .!= length.(keep_variants)
write_fasta("Logging/trimmed_pairs.fasta",permutedims([keep_variants[got_trimmed] trimmed_keep_variants[got_trimmed]])[:])

In [ ]:
keep_freqs = logmarginal_freqs[keep_inds,:]
keep_enrich = logmarginal_enrich[keep_inds,:]
keep_counts = marginal_counts[keep_inds,:];

In [ ]:
tots = Int64.(sum(keep_counts,dims = 2)[:]);
unique_nums = sortperm(sortperm(tots,rev=true));
newnames = [a[3] for a in AA_regions] .* "_" .* string.(unique_nums) .* "_" .* string.(tots);

In [ ]:
new_df = DataFrame()
new_df.names = newnames
new_df.size = tots
new_df.nuc = trimmed_keep_variants
new_df.AA = trimmed_keep_AAs
for i in 1:length(region_names)
    new_df[!, "AA_$(region_names[i])"] = [a[i] for a in AA_regions]
end

new_df.AA_CDR3_length = length.(new_df.AA_CDR3);

#Now we get the nuc CDR3 in here, whcih is required for lineage calling.
for i in 1:length(region_names)
    new_df[!, "nuc_$(region_names[i])"] = [a[i] for a in nuc_regions]
end

for i in 1:length(alldatasets)
    new_df[!, "count_$(alldatasets[i])"] = keep_counts[:,i]
end

for i in 1:length(alldatasets)
    new_df[!, "enrich_$(alldatasets[i])"] = keep_enrich[:,i]
end

for i in 1:length(alldatasets)
    new_df[!, "freq_$(alldatasets[i])"] = keep_freqs[:,i]
end

In [ ]:
t1 = time()
proj = sequmap(new_df.nuc, 2);
new_df.UMAP1 = proj[1,:]
new_df.UMAP2 = proj[2,:]

dim15proj = sequmap(new_df.nuc, 2, pca_maxoutdim = 15);
new_df.dim15UMAP1 = dim15proj[1,:]
new_df.dim15UMAP2 = dim15proj[2,:]

PCAembedding = seqPCA(new_df.nuc);
new_df.PCA1 = PCAembedding[1,:]
new_df.PCA2 = PCAembedding[2,:]
println("Embedding: ", time()-t1, " seconds")

In [ ]:
#KD-Tree Lineage assignment
t1 = time()
#This is using the default seqUMAP projection as the basis for lineage calling. #This makes better lineage plots...
data = permutedims([new_df.UMAP1 new_df.UMAP2])
#but I suspect that using the PCAembedding will give more complete lineages.
#data = PCAembedding
kdtree = KDTree(data)
#This control the radius in which lineages neighbours are searched.
#If you switch to PCA embedding above, you'll need a larger value here - like 1.something
embedded_dist_cutoff = 0.25 
range_inds = inrange(kdtree, data, embedded_dist_cutoff);
println("Edges to check: ",sum(length.(range_inds)))
println("NN graph took: ", time()-t1, " seconds")

In [ ]:
t1 = time()
#To handle missing CDR3s
function kmer_wrap(s)
    if length(s) < 4
        return zeros(UInt32,256)
    else
        return kmer_count(s,4)
    end
end

CDR3lengths = length.(new_df.AA_CDR3); 
CDR3kmers = kmer_wrap.(new_df.nuc_CDR3); 
CDR3kmermat = Array{Float64,2}(hcat(CDR3kmers...));

#I could add a threshold on the rest of the seq in here.
#Then the PCA would just be used to structure the search.
#And the inclusion itself would be on kmer distances in the original space.
CDR3kmer_dist_threshold_eq_len = 0.125#0.15
CDR3kmer_dist_threshold_uneq_len = 0.1
g = SimpleGraph(size(data)[2]);
for v1 in 1:length(range_inds)
    for v2 in range_inds[v1]
        if v1 != v2
            d = corrected_kmer_dist(CDR3kmermat[:,v1],CDR3kmermat[:,v2])
            if CDR3lengths[v1] == CDR3lengths[v2]
                if d < CDR3kmer_dist_threshold_eq_len
                    if !has_edge(g, v1, v2)
                        add_edge!(g, v1, v2)
                    end
                end
            else
                if d < CDR3kmer_dist_threshold_uneq_len
                    if !has_edge(g, v1, v2)
                        add_edge!(g, v1, v2)
                    end
                end
            end
        end
    end
end

lineages = connected_components(g)
lineage_sizes = length.(lineages) #Number of unique sequences in the lineage across all datasets.
lineage_ordering = sortperm(lineage_sizes,rev = true)
lineage_numbers = zeros(Int64,size(new_df)[1]);
for i in 1:length(lineage_ordering)
    lineage_numbers[lineages[lineage_ordering[i]]] .= i
end
new_df.lineage = lineage_numbers;
println("Lineage assignment took: ", time()-t1, " seconds")

In [ ]:
CSV.write("Collected/Variants.csv",new_df);

## Phase III: Visualization
This plots the variant data in multiple ways, and allows selecting candidates for synthesis.

In [ ]:
new_df = CSV.read("Collected/Variants.csv",DataFrame);

In [ ]:
lineage_sizes_in_pre = [sum(new_df.count_TyBaseline[new_df.lineage .== l]) for l in sort(union(new_df.lineage))];

In [ ]:
figure(figsize = (6,3))
bar(1:50,sort(lineage_sizes_in_pre,rev=true)[1:50], 0.5, color = "blue")
#yscale("log")
xlim(0,51)
xlabel("Lineage (ordered by size)")
ylabel("Count")
tight_layout()
savefig("Plots/BaselineLineageSize.pdf")

In [ ]:
function get_hull(emb)
    v = [[emb[1,i],emb[2,i]] for i in 1:size(emb)[2]]
    hull = convex_hull(v)
    return hcat(hull...)
end

#colorlist = ["#e6194b", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", 
#    "#46f0f0", "#f032e6", "#bcf60c", "#fabebe", "#008080", "#e6beff", "#9a6324", 
#    "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000075", "#808080"];#, "#ffffff", "#000000"];

colorlist = ["#e6194b", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", 
    "#46f0f0", "#f032e6", "#bcf60c", "#fabebe", "#008080", "#e6beff", "#9a6324", 
    "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#808080"];

In [ ]:
for emb in ["PCA","UMAP","dim15UMAP"]
    embedding = permutedims([new_df[!,"$(emb)1"] new_df[!,"$(emb)2"]])

    f = figure(figsize = (10.7,11))
    f.set_facecolor("black")
    axis("off")

    #scatter(embedding[1,:],embedding[2,:],color="black",alpha = 0.3,s = big_df.sizes .* 0.08 .+ 0.05, linewidth = 0.0)
    #xl = minimum(embedding[1,:]) - 1.0, maximum(embedding[1,:]) + 1.0
    #yl = minimum(embedding[2,:]) - 1.0, maximum(embedding[2,:]) + 1.0
    scatter(embedding[1,:],embedding[2,:],color="white",alpha = 0.3,s = sqrt.(new_df.size) .* 0.18 .+ 0.05, linewidth = 0.0)
    for i in 1:15
        col = colorlist[mod(i,length(colorlist))+1]
        lin = findall(new_df.lineage .== i)
        hull = get_hull(embedding[:,lin])
        plt.fill(hull[1,:],hull[2,:], alpha = 0.2, color = col, linewidth = 0.0, joinstyle = "round")
        plt.fill(hull[1,:],hull[2,:], alpha = 0.35,linewidth = 1.0, edgecolor=col, fill=false, joinstyle = "round")
        scatter(embedding[1,lin],embedding[2,lin],color=col,alpha = 1.0,s = sqrt.(new_df.size[lin]) .* 0.18 .+ 0.05, linewidth = 0.0)
    end

    for i in 1:15
        #lin = lineages[lineage_ordering[i]]
        lin = findall(new_df.lineage .== i)
        mu_x = mean(embedding[1,lin])
        mu_y = mean(embedding[2,lin])
        annotate(string(i), xy=(mu_x,mu_y),fontsize=6, color = "white")
    end


    #xlim(xl)
    #ylim(yl)
    xlabel("seqUMAP1")
    ylabel("seqUMAP2")
    #=
    cbar = colorbar(fraction=0.026, pad=0.04)
    cbar.set_label("NA")
    cbar.ax.yaxis.set_tick_params(color="black",width=2, length=5)
    cbar.set_ticks([-1.5,0,1.5])
    cbar.set_ticklabels([-1.5,0,1.5])
    =#
    tight_layout()
    savefig("Plots/Lineages_$(emb).png", dpi = 400)
end


In [ ]:
for l in 1:100
    inds = new_df.lineage .== l
    write_fasta("Lineages/lineage_$(l).fasta",new_df.nuc[inds], names = new_df.names[inds])
end

In [ ]:
conditions = [n for n in names(new_df) if length(n) >= 7 && n[1:7] == "enrich_" && n != "enrich_"*yaml["baseline"]]

In [ ]:
toplot = conditions
push!(toplot,"AA_CDR3_length")
for emb in ["PCA","UMAP","dim15UMAP"]
    for tp in toplot
        f = figure(figsize = (12,11))
        f.set_facecolor("black")
        #style.use("dark_background")
        axis("off")
        #scatter(proj[1,:],proj[2,:], s = sqrt.(new_df.size ./ 100), alpha = 0.7, c = clamp.(new_df[:,tp],0,2), cmap = "rainbow")


        colo = new_df[:,tp]
        plotorder = sortperm(colo)
        colo = clamp.(colo,-1.5,Inf)
        scatter(new_df[!,"$(emb)1"][plotorder],new_df[!,"$(emb)2"][plotorder],c = colo[plotorder],cmap = "jet", s = sqrt.(new_df.size[plotorder]) .* 0.18 .+ 0.05, linewidth = 0.0)
        cbar = colorbar(fraction=0.026, pad=0.04)
        cbar.set_label(tp, color = "white")
        cbar.ax.yaxis.set_tick_params(color="white",width=2, length=5)
        cbar.set_ticks([-1.5,0,1.5])
        cbar.set_ticklabels([-1.5,0,1.5])
        tight_layout()
    #    for i in labelled_inds
    #        annotate(new_df.special_labels[i], xy=(new_df.UMAP1[i],new_df.UMAP2[i]),fontsize=8)
    #    end
        savefig("Plots/$(tp)_$(emb).png", dpi = 450)
    end
end

In [ ]:
select_from = ["enrich_"*d["label"] for d in yaml["select_candidates_from"]]

In [ ]:
#Rotate through enrichments and make an automated selection
choices = Int64[]
c = 1
for i in 1:33
    for condition in select_from
        choices = prioritize(new_df[:,condition], c, previously_selected = choices, AAs = new_df.AA, lineages = new_df.lineage, CDR3s = new_df.AA_CDR3);
        c += 1
    end
end

write_fasta("Selections/Selected.fasta",new_df.nuc[choices],names = new_df.names[choices] .* "-" .* string.(new_df.lineage[choices]))
CSV.write("Selections/SelectedVariants.csv",sort(new_df[choices,:]));

In [ ]:
#Hardcoded choice
x = select_from[1]
y = select_from[2]
figure(figsize = (7,7))
scatter(new_df[:,x],new_df[:,y],c = "blue",cmap = "jet", s = sqrt.(new_df.size) .* 0.08 .+ 1.0, linewidth = 0.0, alpha = 0.1)
scatter(new_df[choices,x],new_df[choices,y],c = "red",cmap = "jet", s = sqrt.(new_df.size[choices]) .* 0.08 .+ 1.0, linewidth = 0.5)
xlabel(x)
ylabel(y)
savefig("Plots/Selected_EnrichCoords.png", dpi = 450)

In [ ]:
for emb in ["PCA","UMAP","dim15UMAP"]    
    f = figure(figsize = (10.7,11))
    f.set_facecolor("black")
    axis("off")
    colo = new_df.size
    plotorder = sortperm(colo)
    colo = clamp.(colo,-1.5,Inf)
    scatter(new_df[!,"$(emb)1"][plotorder],new_df[!,"$(emb)2"][plotorder],c = "blue", alpha = 1.0, s = sqrt.(new_df.size[plotorder]) .* 0.08 .+ 0.05, linewidth = 0.0)
    scatter(new_df[!,"$(emb)1"][choices],new_df[!,"$(emb)2"][choices],c = "red", s = sqrt.(new_df.size[choices]) .* 0.08 .+ 2.0, linewidth = 0.0)
    for i in choices
        annotate(string(new_df.lineage[i]), xy=(new_df[!,"$(emb)1"][i],new_df[!,"$(emb)2"][i]),fontsize=8, color = "white")
    end
    tight_layout()
    savefig("Plots/Selected_$(emb).png", dpi = 450)
end

In [ ]:
#If you want to find a specific sequence by AA
#=
query = "GGLVQPGGSLRLSCAASGFTFSSVYMNWVRQAPGKGPEWVSRISPNSGNIGYTDSVKGRFTISRDNAKNTLYLQMNNLKPEDTALYYCAIGLNLSSSSVRGQGTQVTVSS";
query_inds = findall(new_df.AA .== query)
query_pos = query_inds[argmax(new_df.size[query_inds])]
query_df = DataFrame()
query_df.Label = ["Ty1"]
query_df.nuc_sequence = [new_df.nuc[query_pos]]
CSV.write("special.csv",query_df)
=#

In [ ]:
if isfile(yaml["variant_whitelist"])
    special_df = CSV.read(yaml["variant_whitelist"],DataFrame);
    special_dict = Dict(zip(special_df.nuc_sequence,special_df.Label))
    special_labels = [get(special_dict,n,"") for n in new_df.nuc];
    labelled_inds = findall(special_labels .!= "");
    new_df.special_labels = special_labels
else
    new_df.special_labels = ["" for i in 1:size(new_df)[1]]
    println("No whitelist")
end;

In [ ]:
if isfile(yaml["variant_whitelist"])
    toplot = conditions
    push!(toplot,"AA_CDR3_length")
    for emb in ["PCA","UMAP","dim15UMAP"]
        for tp in toplot
            f = figure(figsize = (12,11))
            f.set_facecolor("black")
            #style.use("dark_background")
            axis("off")
            #scatter(proj[1,:],proj[2,:], s = sqrt.(new_df.size ./ 100), alpha = 0.7, c = clamp.(new_df[:,tp],0,2), cmap = "rainbow")


            colo = new_df[:,tp]
            plotorder = sortperm(colo)
            colo = clamp.(colo,-1.5,Inf)
            scatter(new_df[!,"$(emb)1"][plotorder],new_df[!,"$(emb)2"][plotorder],c = colo[plotorder],cmap = "jet", s = sqrt.(new_df.size[plotorder]) .* 0.18 .+ 0.05, linewidth = 0.0)
            cbar = colorbar(fraction=0.026, pad=0.04)
            cbar.set_label(tp, color = "white")
            cbar.ax.yaxis.set_tick_params(color="white",width=2, length=5)
            cbar.set_ticks([-1.5,0,1.5])
            cbar.set_ticklabels([-1.5,0,1.5])
            tight_layout()
            for i in findall(new_df.special_labels .!= "")
                annotate(new_df.special_labels[i], xy=(new_df[!,"$(emb)1"][i],new_df[!,"$(emb)2"][i]),fontsize=8, color = "white")
            end
            savefig("Plots/$(tp)_$(emb)_Labelled.png", dpi = 450)
        end
    end
else
    println("No whitelist")
end